# Serverless

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [2]:
!wget https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing_model.h5 --quiet

In [4]:
model = keras.models.load_model('clothing_model.h5')

In [5]:
!wget http://bit.ly/mlbookcamp-pants -O pants.jpg --quiet

In [7]:
img = load_img('pants.jpg', target_size=(299, 299))

In [8]:
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

In [9]:
X.shape

(1, 299, 299, 3)

In [10]:
preds = model.predict(X)
preds

1/1 [==============================] - 1s 1s/step


array([[-1.8682904, -4.7612453, -2.3169832, -1.0625703,  9.887159 ,
        -2.8124344, -3.666283 ,  3.2003617, -2.602338 , -4.835046 ]],
      dtype=float32)

In [11]:
labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

results = dict(zip(labels, preds[0]))
results

{'dress': -1.8682904,
 'hat': -4.7612453,
 'longsleeve': -2.3169832,
 'outwear': -1.0625703,
 'pants': 9.887159,
 'shirt': -2.8124344,
 'shoes': -3.666283,
 'shorts': 3.2003617,
 'skirt': -2.602338,
 't-shirt': -4.835046}

# Convert Keras to TF-Lite

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with tf.io.gfile.GFile('clothing-model.tflite', 'wb') as f:
    f.write(tflite_model)